In [1]:
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.linear_model import LinearRegression

In [2]:
init_notebook_mode(connected = True)

### Эталонные растворы

In [3]:
n_ref = np.array([98.8, 71.7, 44.4, 35.4, 30.4]) # показания спектрофотометра
neg_log_n_ref = -np.log(n_ref / 100) # -ln(n / 100)

C_I2_H2O_ref = np.array([0.03, 0.28, 0.70, 0.91, 1.23]) * 1e-4 # концентрации эталонных растворов

### Исследуемые растворы

In [4]:
n = np.array([66., 52.4, 46.2]) # показания спектрофотометра
neg_log_n = -np.log(n / 100) # -ln(n / 100)

C_I2_CCl4 = np.array([0.61, 1.08, 1.58]) * 1e-2 # концентрации исследуемых растворов

### Аппроксимирующая прямая (МНК)

In [5]:
reg = LinearRegression(fit_intercept = False)

reg.fit(neg_log_n_ref.reshape(-1, 1), C_I2_H2O_ref)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

### Равновесные концентрации исследуемых растворов C<sup>равн</sup><sub>I<sub>2</sub> в H<sub>2</sub>O

In [6]:
C_I2_H2O_bal = reg.predict(neg_log_n.reshape(-1, 1)) # определяем значения равновесных концентраций по аппроксимирующей прямой
print(C_I2_H2O_bal)

[3.90731069e-05 6.07715716e-05 7.26131316e-05]


In [7]:
C_I2_H2O_bal = np.around(C_I2_H2O_bal, decimals = 6) # округляем значения
print(C_I2_H2O_bal)

[3.9e-05 6.1e-05 7.3e-05]


In [8]:
print('Равновесные коцентрации исследуемых растворов I2 в H2O')
for i in range(len(neg_log_n)):
    print('n = {0:2f}, C = {1}'.format(neg_log_n[i], C_I2_H2O_bal[i]))

Равновесные коцентрации исследуемых растворов I2 в H2O
n = 0.415515, C = 3.9e-05
n = 0.646264, C = 6.1e-05
n = 0.772190, C = 7.3e-05


### График зависимости логарифма показаний прибора -ln(n / 100) от концентрации раствора C<sub>I<sub>2</sub> в H<sub>2</sub>O

In [9]:
k = reg.coef_ # C = -ln(n / 100) * k

C_line = np.array([0, 130e-6])
n_log_line = C_line / reg.coef_

In [10]:
p1 = go.Scatter(x = C_I2_H2O_ref, y = neg_log_n_ref, mode = 'markers', hoverinfo = 'x+y',
                marker = dict(color = 'rgb(0, 128, 128)'), name = 'Эталонные растворы')

p2 = go.Scatter(x = C_line, y = n_log_line, mode = 'lines', 
                               line = dict(color = 'rgb(135, 206, 235)', width = 3), name = 'Аппроксимирующая прямая')

p3 = go.Scatter(x = C_I2_H2O_bal, y = neg_log_n, mode = 'markers', hoverinfo = 'x+y',
                marker = dict(color = 'rgb(178, 34, 34)'), name = 'Исследуемые растворы')

layout = go.Layout(title = 'График зависимости логарифма показаний прибора -ln(n / 100) от концентрации раствора C',
                   xaxis = dict(showgrid = True, zeroline = True),
                   yaxis = dict(showgrid = True, zeroline = True),
                   showlegend = True, hovermode = 'closest')

fig = go.Figure(data = [p1, p2, p3], layout = layout)

fig.update_yaxes(tickvals = np.linspace(0, 1.2, 7), hoverformat = '.2f')
fig.update_xaxes(tickvals = np.linspace(0, 120e-6, 7), hoverformat = '.6f')

py.offline.iplot(fig)

### Равновесные концентрации C<sup>равн</sup><sub>I<sub>2</sub> в CCl<sub>4

In [11]:
C_I2_CCl4_bal = C_I2_CCl4 - (30 / 2) * C_I2_H2O_bal

In [12]:
print('Равновесные коцентрации исследуемых растворов I2 в CCl4')
for i in range(len(neg_log_n)):
    print('C = {0:f}'.format(C_I2_CCl4_bal[i]))

Равновесные коцентрации исследуемых растворов I2 в CCl4
C = 0.005515
C = 0.009885
C = 0.014705


### Проверка закона распределения

In [13]:
print('L = C_I2_H2O_bal / C_I2_CCl4_bal')
for i in range(len(C_I2_CCl4)):
    print('L{0} = {1:3f} / {2:3f} = {3:4f}'.format(i + 1, C_I2_H2O_bal[i], C_I2_CCl4_bal[i], C_I2_H2O_bal[i] / C_I2_CCl4_bal[i]))

L = C_I2_H2O_bal / C_I2_CCl4_bal
L1 = 0.000039 / 0.005515 = 0.007072
L2 = 0.000061 / 0.009885 = 0.006171
L3 = 0.000073 / 0.014705 = 0.004964
